In [ ]:
from os import getenv

input_folder = getenv('CROSSCOMPUTE_INPUT_FOLDER', 'batches/standard/input')
output_folder = getenv('CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output')
log_folder = getenv('CROSSCOMPUTE_LOG_FOLDER', 'batches/standard/log')

In [ ]:
from os.path import join

log_file = open(join(log_folder, 'log.txt'), 'wt')

In [ ]:
import geotable

region_path = join(input_folder, 'region.geojson')
region_geotable = geotable.load(region_path)
try:
    region_proj4 = region_geotable['geometry_proj4'][0]
except (KeyError, IndexError):
    log_file.write('No region specified. Please specify at least one region.')
region_geotable

In [ ]:
from shapely.ops import unary_union

region_geometry = unary_union(region_geotable.geometries)
region_geometry

In [ ]:
import json

settings = json.load(open(join(input_folder, 'variables.dictionary')))
settings

In [ ]:
point_count = settings['count']
point_count

In [ ]:
import random
from shapely.geometry import Point
from shapely.ops import unary_union

def make_random_points(region_geometry, target_count):
    points = []
    minimum_x, minimum_y, maximum_x, maximum_y = region_geometry.bounds
    while len(points) < target_count:
        # Generate random points inside bounds
        random_points = [Point(
            random.uniform(minimum_x, maximum_x),
            random.uniform(minimum_y, maximum_y),
        ) for _ in range(target_count)]
        # Retain points inside region
        collection = unary_union(random_points + points)
        intersection = collection.intersection(region_geometry)
        if intersection.type == 'Point':
            points = [intersection]
        else:
            points = list(intersection.geoms)
    # Trim if there are too many
    return points[:target_count]

from shapely.geometry import GeometryCollection
points = make_random_points(region_geometry, point_count)
GeometryCollection(points + [region_geometry])

In [ ]:
from geotable import ColorfulGeometryCollection
ColorfulGeometryCollection(points)

In [ ]:
points_geotable = geotable.GeoTable()
points_geotable['geometry_object'] = points
points_geotable['geometry_proj4'] = region_proj4
points_geotable.save_geojson(join(output_folder, 'points.geojson'))